## Process Data

In [1]:
import h5py 

file = h5py.File("datasets\postera_protease2_pos_neg_train.hdf5","r")
ligand_names = list(file.keys())

ligands= [] 
labels= [] 

for name in ligand_names:
    
    # use enumeration to read in information
    ligand = file[name]["ligand"]
    label = file[name].attrs["label"]
    atom_coordinates = ligand[:,:3]
    atom_features = ligand[:,3:]
    
    # collect
    ligands.append(ligand)
    labels.append(label)
 
    


In [2]:
ligand_names = list(file.keys())
index = 10 
name = ligand_names[10]
ligand = file[name]["ligand"]


label = file[name].attrs["label"]
coordinates = ligand[:,:3]
features = ligand[:,3:]

#print(ligand[:])


## Make Dataset Object

Make Dataset object to load in ligand data and labels while allowing mulit-processing. 

In [3]:
import torch 
from torch.utils.data import Dataset
import h5py

class LigandDataset(Dataset):
    """
    Custom Dataset object that opens the file path for the postera_protease2 datasets. 
    """
    def __init__(self, hdf_path, parse_features=False):
        super().__init__()
        self.hdf_path = hdf_path
        self.file = h5py.File(self.hdf_path,"r")
        self.ligand_names = list(self.file.keys())
        self.parse_features = parse_features
        
    def __len__(self):
        """ 
        Returns the total number of ligands. 
        """
        return len(self.ligand_names)
        
    def __getitem__(self, index):
        """
        Returns coordinates and/or features as well as labels for each ligand pose.
        """
        name = self.ligand_names[index]
        ligand = self.file[name]["ligand"]
        label = self.file[name].attrs["label"]
        
        
        if self.parse_features:
            coordinates = ligand[:,:3]
            features = ligand[:,3:]
            return coordinates, features, label
        else:
            return ligand[:], label
    

In [4]:
import os
import sys
import math
import numbers
import numpy as np
import scipy as sp
import torch
import torch.nn as nn


class Voxelizer3D(nn.Module):
    """
    Orignal Voxelizer from FAST fusion model. Note that data can only be passed in one-at-a-time rather than in batches. 
    """
    def __init__(self, feat_dim=19, vol_dim=48, ang_size=48, relative_size=True, atom_radius=1, use_cuda=True, verbose=0):
        super(Voxelizer3D, self).__init__()

        self.feat_dim = feat_dim
        self.vol_dim = vol_dim
        self.ang_size = ang_size
        self.relative_size = relative_size
        self.atom_radius = atom_radius
        self.use_cuda = use_cuda
        self.verbose = verbose

    def forward(self, xyz, feat, atom_radii=None):
        # filter out zero padded rows
        mask = (xyz[:,0] != 0) & (xyz[:,1] != 0) & (xyz[:,2] != 0)
        xyz = xyz[mask]
        feat = feat[mask]

        # get 3d bounding box
        xmin, ymin, zmin = min(xyz[:,0]), min(xyz[:,1]), min(xyz[:,2])
        xmax, ymax, zmax = max(xyz[:,0]), max(xyz[:,1]), max(xyz[:,2])
        if self.relative_size:
            # voxel size (assuming voxel size is the same in all axis)
            vox_size = float(zmax - zmin) / float(self.vol_dim)
        else:
            vox_size = float(self.ang_size) / float(self.vol_dim)
            xmid, ymid, zmid = (xmin + xmax) / 2.0, (ymin + ymax) / 2.0, (zmin + zmax) / 2.0
            xmin, ymin, zmin = xmid - (self.ang_size / 2), ymid - (self.ang_size / 2), zmid - (self.ang_size / 2)
            xmax, ymax, zmax = xmid + (self.ang_size / 2), ymid + (self.ang_size / 2), zmid + (self.ang_size / 2)

        # initialize vol data
        if self.use_cuda:
            vol_data = torch.cuda.FloatTensor(self.vol_dim, self.vol_dim, self.vol_dim, self.feat_dim).fill_(0)
        else:
            vol_data = torch.zeros((self.vol_dim, self.vol_dim, self.vol_dim, self.feat_dim)).float()

        # assign each atom to voxels
        for ind in range(xyz.shape[0]):
            x, y, z = xyz[ind, 0], xyz[ind, 1], xyz[ind, 2]
            if x < xmin or x > xmax or y < ymin or y > ymax or z < zmin or z > zmax:
                continue

            # compute van der Waals radius and atomic density, use 1 if not available
            if not atom_radii is None:
                vdw_radius = atom_radii[ind]
                atom_radius = 1 + vdw_radius * vox_size
            else:
                atom_radius = self.atom_radius

            cx = int((x-xmin) / (xmax-xmin) * (self.vol_dim-1))
            cy = int((y-ymin) / (ymax-ymin) * (self.vol_dim-1))
            cz = int((z-zmin) / (zmax-zmin) * (self.vol_dim-1))
            vx_from = max(0, int(cx-atom_radius))
            vx_to = min(self.vol_dim-1, int(cx+atom_radius))
            vy_from = max(0, int(cy-atom_radius))
            vy_to = min(self.vol_dim-1, int(cy+atom_radius))
            vz_from = max(0, int(cz-atom_radius))
            vz_to = min(self.vol_dim-1, int(cz+atom_radius))

            vol_feat = feat[ind,:].repeat(vz_to-vz_from+1, vy_to-vy_from+1, vx_to-vx_from+1, 1)
            vol_data[vz_from:vz_to+1, vy_from:vy_to+1, vx_from:vx_to+1, :] += vol_feat

        #vol_data = vol_data.permute(3,0,1,2) #-> doesn't need as we already initialized 19x48x48x48
        return vol_data
    
    
class GaussianFilter(nn.Module):
    def __init__(self, dim=2, channels=3, kernel_size=11, sigma=1, use_cuda=True):
        super(GaussianFilter, self).__init__()

        self.use_cuda = use_cuda
        if isinstance(kernel_size, numbers.Number):
            self.kernel_size = [kernel_size] * dim
        if isinstance(sigma, numbers.Number):
            self.sigma = [sigma] * dim

        self.padding = kernel_size // 2

        # Gaussian kernel is the product of the gaussian function of each dimension.
        kernel = 1
        meshgrids = torch.meshgrid([torch.arange(size, dtype=torch.float32) for size in self.kernel_size])
        for size, std, mgrid in zip(self.kernel_size, self.sigma, meshgrids):
            mean = (size - 1) / 2
            kernel *= 1 / (std * math.sqrt(2 * math.pi)) * torch.exp(-((mgrid - mean) / std) ** 2 / 2)

        # make sure sum of values in gaussian kernel equals 1.
        kernel = kernel / torch.sum(kernel)

        # reshape to depthwise convolutional weight
        kernel = kernel.view(1, 1, *kernel.size()) #-> doesn't need to add one more axis
        #kernel = kernel.view(1, *kernel.size())
        kernel = kernel.repeat(channels, *[1] * (kernel.dim() - 1))
        if self.use_cuda:
            kernel = kernel.cuda()

        self.register_buffer('weight', kernel)
        self.groups = channels
        if dim == 1:
            self.conv = nn.functional.conv1d
        elif dim == 2:
            self.conv = nn.functional.conv2d
        elif dim == 3:
            self.conv = nn.functional.conv3d

    def forward(self, input):
        return self.conv(input, weight=self.weight, groups=self.groups, padding=self.padding)

In [5]:
class Model_3DCNN(nn.Module):

    # num_filters=[64,128,256] or [96,128,128]
    def __init__(self, feat_dim=19, output_dim=1, num_filters=[64,128,256], use_cuda=True, verbose=0):
        super(Model_3DCNN, self).__init__()

        self.feat_dim = feat_dim
        self.output_dim = output_dim
        self.num_filters = num_filters,
        self.use_cuda = use_cuda
        self.verbose = verbose

        self.conv_block1 = self.__conv_layer_set__(self.feat_dim, self.num_filters[0], 7, 2, 3)
        self.res_block1 = self.__conv_layer_set__(self.num_filters[0], self.num_filters[0], 7, 1, 3)
        self.res_block2 = self.__conv_layer_set__(self.num_filters[0], self.num_filters[0], 7, 1, 3)

        self.conv_block2 = self.__conv_layer_set__(self.num_filters[0], self.num_filters[1], 7, 3, 3)
        self.max_pool2 = nn.MaxPool3d(2)

        self.conv_block3 = self.__conv_layer_set__(self.num_filters[1], self.num_filters[2], 5, 2, 2)
        self.max_pool3 = nn.MaxPool3d(2)

        self.fc1 = nn.Linear(2048, 100)
        torch.nn.init.normal_(self.fc1.weight, 0, 1)
        self.fc1_bn = nn.BatchNorm1d(num_features=100, affine=True, momentum=0.1).train()
        self.fc2 = nn.Linear(100, 1)
        torch.nn.init.normal_(self.fc2.weight, 0, 1)
        self.relu = nn.ReLU()
        #self.drop=nn.Dropout(p=0.15)

    def __conv_layer_set__(self, in_c, out_c, k_size, stride, padding):
        conv_layer = nn.Sequential(
            nn.Conv3d(in_c, out_c, kernel_size=k_size, stride=stride, padding=padding, bias=True),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(out_c))
        return conv_layer

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(-1)
        conv1_h = self.conv_block1(x)
        if self.verbose != 0:
            print(conv1_h.shape)

        conv1_res1_h = self.res_block1(conv1_h)
        if self.verbose != 0:
            print(conv1_res1_h.shape)

        conv1_res1_h2 = conv1_res1_h + conv1_h
        if self.verbose != 0:
            print(conv1_res1_h2.shape)

        conv1_res2_h = self.res_block2(conv1_res1_h2)
        if self.verbose != 0:
            print(conv1_res2_h.shape)

        conv1_res2_h2 = conv1_res2_h + conv1_h
        if self.verbose != 0:
            print(conv1_res2_h2.shape)

        conv2_h = self.conv_block2(conv1_res2_h2)
        if self.verbose != 0:
            print(conv2_h.shape)

        pool2_h = self.max_pool2(conv2_h)
        if self.verbose != 0:
            print(pool2_h.shape)

        conv3_h = self.conv_block3(pool2_h)
        if self.verbose != 0:
            print(conv3_h.shape)

        pool3_h = conv3_h
        #pool3_h = self.max_pool3(conv3_h)
        #if self.verbose != 0:
        #	print(pool3_h.shape)

        flatten_h = pool3_h.view(pool3_h.size(0), -1)
        if self.verbose != 0:
            print(flatten_h.shape)

        fc1_z = self.fc1(flatten_h)
        fc1_y = self.relu(fc1_z)
        fc1_h = self.fc1_bn(fc1_y) if fc1_y.shape[0]>1 else fc1_y  #batchnorm train require more than 1 batch
        if self.verbose != 0:
            print(fc1_h.shape)

        fc2_z = self.fc2(fc1_h)
        if self.verbose != 0:
            print(fc2_z.shape)

        return fc2_z, fc1_z

In [6]:
# Define voxelizer and gaussian filter objects
use_cuda = True
device = "cuda"
voxelizer = Voxelizer3D(use_cuda=use_cuda,verbose=0)
gaussian_filter = GaussianFilter(dim=3, channels=19, kernel_size=11, sigma=1, use_cuda=use_cuda)
model = Model_3DCNN(use_cuda=True)

c:\Users\Killian\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


TypeError: unsupported operand type(s) for %: 'list' and 'int'

In [81]:
# test by passing through a single batch
path = "datasets\postera_protease2_pos_neg_train.hdf5"
train_data = LigandDataset(path,parse_features=False)
train_dataloader = DataLoader(train_data, batch_size=batch_size,shuffle=False)

# initialize batch data
vol_batch = torch.zeros( (batch_size,19,48,48,48), dtype=torch.float, device=device)
model.to(device)

batch_x, batch_y = next(iter(train_dataloader))
inputs, labels = batch_x.to(device), batch_y.to(device)

for i in range(inputs.shape[0]):
    xyz, feat = inputs[i,:,:3], inputs [i,:,3:]
    vol_batch[i,:,:,:,:] = voxelizer(xyz,feat)
    
vol_batch = gaussian_filter(vol_batch)
y_pred_batch, _ = model(vol_batch)

# for batch_idx, batch_data in enumerate(train_dataloader):
    
#     inputs_cpu, labels_cpu = batch_data
#     inputs, labels = inputs_cpu.to(device), labels_cpu.to(device)
    
    
#     for i in range(inputs.shape[0]):
#         xyz, feat = inputs[i,:,:3], inputs [i,:,3:]
#         vol_batch[i,:,:,:,:] = voxelizer(xyz,feat)
#     print(vol_batch.shape)
#     vol_batch = gaussian_filter(vol_batch)

# xyz, feat = check_feature[:,:3], check_feature[:,3:]
# print(xyz.shape, feat.shape)
# vol = voxelizer(xyz,feat)
# print(vol.shape)
print(vol_batch.shape)
print(y_pred_batch)

torch.Size([10, 19, 48, 48, 48])
tensor([[ 13.9965],
        [ -6.3637],
        [-12.7872],
        [ 33.4972],
        [-15.9389],
        [ 12.6863],
        [  5.3891],
        [-29.9800],
        [ 10.7067],
        [-11.9602]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [65]:
print(vol_batch[0][0][0][0])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')


In [83]:
# import time
# from torch.utils.data import DataLoader

# # set parameters
# device = torch.device("cuda")
# batch_size = 10


# # Load data
# start_load = time.time()
# path = "datasets\postera_protease2_pos_neg_train.hdf5"
# train_data = LigandDataset(path,parse_features=False)
# train_dataloader = DataLoader(train_data, batch_size=batch_size,shuffle=False)
# end_load = time.time()
# print(f"Program loaded data in {end_load-start_load} seconds")

# # initialize volume information
# start_voxel = time.time()
# vol_batch = torch.zeros( (batch_size,48,48,48,19), dtype=torch.float, device=device)
# for batch_idx, batch_data in enumerate(train_dataloader):
    
#     inputs_cpu, labels_cpu = batch_data
#     inputs, labels = inputs_cpu.to(device), labels_cpu.to(device)
    
    
#     for i in range(inputs.shape[0]):
#         xyz, feat = inputs[i,:,:3], inputs [i,:,3:]
#         vol_batch[i,:,:,:,:] = voxelizer(xyz,feat)
#     print(vol_batch.shape)
#     vol_batch = gaussian_filter(vol_batch)
    
# end_voxel = time.time()


# print(f"Voxelization completed in {end_voxel-start_voxel} seconds")

In [126]:
print(vol_batch)

tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],

          ...,

          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           